## Import Libraries

In [19]:
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum

## Class Definition 

In [20]:
class ServerIDs(Enum):
    """
    Description: 
        Class storing server names
    """
    msn = 'MSN'
    asn_1 = 'ASN1'
    asn_2 = 'ASN2'

class Status(Enum):
    """
    Description: 
        Class storing status - First Come First Serve (FCFS) principle
    """
    handled = -1
    served = 0
    arrived = np.inf
    
class CustomerGroup:
    """
    Description: 
        Class storing all customer group specific information
    """
    def __init__(self,ID,popularity,activity_pattern,distances):
        self.ID = ID
        self.popularity = popularity
        self.weights=popularity/np.sum(popularity)
        self.activity_pattern = activity_pattern
        self.distances = distances
    
    def best_server_options(self):
        """
        Description: 
            Finding server options
        Return:
            list (str) - sorted list of server options from best to worst
        """
        return sorted(self.distances, key=self.distances.get)
     
class Server:
    """ 
    Description: 
        Class representing the servers {MSN, ASN1, ASN2}
    """
    def __init__(self, id_, movies_stored, capacity, movie_sizes):
        self.id = id_
        self.movies_stored = movies_stored
        self.capacity = capacity
        self.movie_sizes = movie_sizes
        
         # Parameter for checking that the capacity limited of the server is not succeeded
        self.capacity_check = capacity
        self.check_capacity_limit()
        
    def check_capacity_limit(self):
        """
        Description: 
            Check capacity limited is not succeeded
        """
        for movie in self.movies_stored:
            self.capacity_check -= self.movie_sizes[movie]
            if self.capacity_check < 0:
                raise Exception('Server Capacity Exceeded')
        
class LoadBalancer:
    """ 
    Description: 
        Superclass storing the servers and all associated information (e.g. such as serving time)
    """
    def __init__(self, movies_stored_msn, movies_stored_asn_1, movies_stored_asn_2,\
                 capacities, serve_times, movie_sizes):
        self.msn = Server(ServerIDs.msn.value, movies_stored_msn, capacities[0], movie_sizes)
        self.asn_1 = Server(ServerIDs.asn_1.value, movies_stored_asn_1, capacities[1], movie_sizes)
        self.asn_2 = Server(ServerIDs.asn_2.value, movies_stored_asn_2, capacities[2], movie_sizes)
        self.serve_times = serve_times
        self.movie_sizes = movie_sizes
        
    def get_serve_time(self, movie: int, group_id: int, server_id: str):
        """
        Description: 
            Return service time based on input arguments
        Args: 
            int - chosen movie
            int - group_id of customer group \in {1,2,3}
            string - server_id of server \in {'MSN', 'ASN1', 'ASN2'}
        Return:
            Int - serve time
        """
        movie_size = self.movie_sizes[movie]
        if movie_size < 900:
            return self.serve_times[900][server_id][group_id]
            
        elif movie_size >= 900 and movie_size < 1100:
            return self.serve_times[1100][server_id][group_id]
            
        elif movie_size >= 1100:
            return self.serve_times[1500][server_id][group_id]
        else:
            raise Exception(f'Movie Size: {movie_size}')
        
class SingleCustomer:
    """ 
    Description: 
        Class representing single customer
    """
    def __init__(self,id_,time,movie_choice,server_address,waiting_time):
        self.id_ = id_
        self.status = Status.arrived.value
        self.time = time
        self.movie_choice = movie_choice
        self.waiting_time = waiting_time
        self.server_address = server_address

## Function Definiton

In [21]:
def exponential_rng(lam=1.0):  
    """ 
    Description:
        Generates exponential random number
    Args:
        float - lam, the rate parameter, the inverse expectation of the distribution
    Return:
        float - Exponential random number with given rate
    """
    return -np.log(np.random.rand()) / lam

def homogeneous_poisson_process(lam, T):
    """ 
    Description:
        Simulate arrivals via homogeneous Poisson process
    Args:
        float - lam, the rate parameter, the inverse expectation of the distribution
        int - simulation period
    Return:
        list (float) - Arrival times
    """
    arrival_times=[]
    curr_arrival_time=0
    while True:
        curr_arrival_time += exponential_rng(lam)
        if curr_arrival_time > T:
            break
        else:
            arrival_times.append(curr_arrival_time)
    return arrival_times

def adjust_time_and_pick_a_movie(arrival_times: list, delta_T: int, G: CustomerGroup, load_balancer: LoadBalancer)-> list:
    """
    Description:
        Create single customers and assign attributes (e.g. movie, server address)
    Args:
        list (float) - arrival times
        int - delta_T, time offset for arrival times
        class - Customer group
        class - Load balancer
    Return:
        list (class) - List of customers
    """
    output = []
    for arrival_time in arrival_times:
        movie = np.random.choice(np.arange(10, step=1), size=None, replace=True, p=G.weights)
        servers_with_movie = check_movie_availability_on_server(movie,load_balancer)
        best_servers = G.best_server_options()
        serverAddress = assign_server(servers_with_movie,best_servers)
        waiting_time = G.distances[serverAddress]
        output.append(SingleCustomer(G.ID, arrival_time + delta_T+waiting_time,\
                                     movie,serverAddress,waiting_time))
    return output

def assign_server(servers_with_movie,best_servers):
    """
    Description:
        Find server assignment for single customer
    Args:
        list (str) - server names that movie is stored on
        list (str) - sorted list of optimal servers
    Return:
        list (class) - List of customers
    """
    for best in best_servers:
        if best in servers_with_movie:
            if best!=np.inf:
                return best
            else:
                raise Exception('Movie not available for customer!')

def check_movie_availability_on_server(movie: int,load_balancer: LoadBalancer):
    """
    Description:
        Find all servers that the selected movie is stored on
    Args:
        int - selected movie by customer
        class - Load balancer
    Return:
        list (str) - server names that movie is stored on
    """        
    servers_with_movie=[]
    if movie in load_balancer.msn.movies_stored:
        servers_with_movie.append(ServerIDs.msn.value)
    if movie in load_balancer.asn_1.movies_stored:
        servers_with_movie.append(ServerIDs.asn_1.value)
    if movie in load_balancer.asn_2.movies_stored:
        servers_with_movie.append(ServerIDs.asn_2.value)
    if servers_with_movie==[]:
        raise Exception('Movie not stored on any server!')
    return servers_with_movie

In [22]:
def generate_customers(G1: CustomerGroup, G2: CustomerGroup, G3: CustomerGroup, LB: LoadBalancer):
    """
    Description:
        Create happy hour demand from three customer groups
    Args:
        class - Customer group 1
        class - Customer group 2
        class - Customer group 3
        class - Load balancer
    Return:
        list (class) - List of customers
    """ 
    
    # Simulation time in seconds
    T = 20 * 60
    
    requests = []
    
    # Generate Arrivals per group via homogeneous Poisson process based on group specific activity pattern
    for activity_number in range(0,3):
        
        # either 0, 20, 40 to make 1 hour of requests
        delta_T = activity_number * 20 * 60
        
        # Time of the events
        arrival_times_1 = homogeneous_poisson_process(G1.activity_pattern[activity_number], T)
        arrival_times_2 = homogeneous_poisson_process(G2.activity_pattern[activity_number], T)
        arrival_times_3 = homogeneous_poisson_process(G3.activity_pattern[activity_number], T)
        
        customers_1 = adjust_time_and_pick_a_movie(arrival_times_1, delta_T, G1, LB)
        customers_2 = adjust_time_and_pick_a_movie(arrival_times_2, delta_T, G2, LB)
        customers_3 = adjust_time_and_pick_a_movie(arrival_times_3, delta_T, G3, LB)
    
        merged_customers = customers_1 + customers_2 + customers_3
        
        merged_customers.sort(key=lambda customers:customers.time)
        
        requests = requests + merged_customers
    return requests

def process_customers(customers, load_balancer):
    """
    Description:
        Process all customers of specified server and calcuate processing times accordingly
    Args:
        list (class) - List of unserved customers
        class - Load balancer
    Return:
        list (class) - List of served customers
    """ 
    
    # First customer
    current_time = customers[0].time
    # List of served customers
    customers_served = []
    
    # either the server is busy or not
    server_busy = False
    server_busy_count = 0
    while len(customers):
        c = customers[0]
        
        # A new request arrives to the server
        # This request has to be "handled"
        # The handle time follows an exponential distribution
        # with the mean of 0.5 second
        if c.status == Status.arrived.value and not server_busy:
            
            server_busy_count = 0
            
            c.status = Status.handled.value
            time_to_handle = exponential_rng(lam=2)
            
            
            if c.time > current_time:
                current_time = c.time

            c.waiting_time += time_to_handle
            current_time += time_to_handle
            c.time += time_to_handle
                
            server_busy = True
            #print('{:<15} {:<15}'.format(f'Time: {round(current_time,2)}s', f'Event: New Arrival'))
        
        # The client request was already handled
        # Now we have to serve the movie
        # The serve time is defined in Table 5 + some noise
        # The noise is uniformly distributed between [0.3, 0.7]
        elif c.status == Status.handled.value and server_busy:
            
            server_busy_count = 0
            movie = c.movie_choice
            group_id = c.id_
            server_id = c.server_address
            time_to_serve = load_balancer.get_serve_time(movie, group_id, server_id)
            time_to_serve += np.random.uniform(0.3, 0.7)
            
            current_time += time_to_serve
            server_busy = False
            
            c.status = Status.served.value
            c.waiting_time += time_to_serve
            customers_served.append(c)
            customers.pop(0)
            #print('{:<15} {:<15}'.format(f'Time: {round(current_time,2)}s', f'Event: Customer Served'))
        
        # A new request arrives but the server is busy
        # update the waiting time and time
        elif c.status == Status.arrived.value and server_busy:
            server_busy_count += 1
            if server_busy_count == 1000:
                breakpoint()
            c.waiting_time += (current_time - c.time)
            c.time = current_time
            #print('{:<15} {:<15}'.format(f'Time: {round(current_time,2)}s', f'Event: Server Busy'))

        customers = sorted(customers, key = lambda customer: (customer.time, customer.status))
        
    return customers_served


def handle_requests():
    """
    Description:
        Simulate customer arrival and request handling process
    Return:
        list (class) - List of served customers
    """ 
    
    # assuming that the ASNs have the same movies stored - Otherwise adjust movies_stored_asn
    load_balancer = LoadBalancer(movies_stored_msn, movies_stored_asn, movies_stored_asn,\
                                 capacities, serve_times, movie_sizes)
    
    G1 = CustomerGroup(1,popularities[0],activity_patterns[0],distances_g1)
    G2 = CustomerGroup(2,popularities[1],activity_patterns[1],distances_g2)
    G3 = CustomerGroup(3,popularities[2],activity_patterns[2],distances_g3)
    
    # Generate customers according to customer groups
    customers = generate_customers(G1,G2,G3,load_balancer)
    
    customers_msn = []
    customers_asn_1 = []
    customers_asn_2 = []
    
    for customer in customers:
        
        if customer.server_address==ServerIDs.msn.value:
            customers_msn.append(customer)
        elif customer.server_address==ServerIDs.asn_1.value:
            customers_asn_1.append(customer)
        elif customer.server_address==ServerIDs.asn_2.value:
            customers_asn_2.append(customer)
    
    # Sorting to ensure order dependent on time
    customers_msn.sort(key=lambda customers:customers.time)
    customers_asn_1.sort(key=lambda customers:customers.time)
    customers_asn_2.sort(key=lambda customers:customers.time)
    
    
    # Get customers per server
    print('='*47)
    print('MSN')
    print('='*47)
    print('{:<30}'.format(f'MSN Customers Length: {len(customers_msn)}'))
    customers_msn = process_customers(customers_msn, load_balancer)
    
    print('='*47)
    print('ASN1')
    print('='*47)
    print('{:<30}'.format(f'ASN1 Customers Length: {len(customers_asn_1)}'))
    customers_asn_1 = process_customers(customers_asn_1, load_balancer)
  
    print('='*47)
    print('ASN2')
    print('='*47)
    print('{:<30}'.format(f'ASN2 Customers Length: {len(customers_asn_2)}'))
    customers_asn_2 = process_customers(customers_asn_2, load_balancer)
    
    # Generate all stats that you want
    return customers_msn + customers_asn_1 + customers_asn_2

def get_statistics(results: list):
    """
    Description:
        Calculate performance indicators / statistics of process
    Args:
        list (class) - List of served customers
    Return:
        dict - Statistics of process
    """ 
    waiting_times = dict()
    waiting_times[ServerIDs.msn.value] = []
    waiting_times[ServerIDs.asn_1.value] = []
    waiting_times[ServerIDs.asn_2.value] = []
    waiting_times[1] = []
    waiting_times[2] = []
    waiting_times[3] = []
    for customer in results:
        
        waiting_time = customer.waiting_time
        waiting_times[customer.server_address].append(waiting_time)    
        waiting_times[customer.id_].append(waiting_time)
    
    
    statistics = dict()
    
    for key_ in waiting_times.keys():
        waiting_list = np.array(waiting_times[key_])
        statistics[key_] = {
            'mean': waiting_list.mean(),
            'std': waiting_list.std(),
            'max': waiting_list.max(),
            'min': waiting_list.min(),
            'median': np.median(waiting_list),
            'q25': np.quantile(waiting_list, 0.25),
            'q75': np.quantile(waiting_list, 0.75)
        }
    return statistics

## Input Definition

In [23]:
capacities = [np.inf, 3500,3500]

movie_sizes = {
    0: 850,
    1: 950,
    2: 1000,
    3: 1200,
    4: 800,
    5: 900,
    6: 1000,
    7: 750,
    8: 700,
    9: 1100
}

movies_stored_msn = list(range(0, 10))
movies_stored_asn = [2, 3,9]

# --- Table 2 ---
# Preferences
popularities = [[2,4,9,8,1,3,5,7,10,6],[6,1,3,4,7,9,2,5,8,10],[4,7,3,6,1,10,2,9,8,5]]

# --- Table 3 ---
# Lambdas
activity_patterns = [[0.8,1.2,0.5],[0.9,1.3,0.3],[0.7,1.5,0.4]]

# --- Table 4 ---
distances_g1 = {ServerIDs.msn.value: 0.5, ServerIDs.asn_1.value: 0.2, ServerIDs.asn_2.value: np.inf}
distances_g2 = {ServerIDs.msn.value: 0.5, ServerIDs.asn_1.value: 0.3, ServerIDs.asn_2.value: 0.4}
distances_g3 = {ServerIDs.msn.value: 0.5, ServerIDs.asn_1.value: np.inf, ServerIDs.asn_2.value: 0.2}

# --- Table 5 ---
# [700- 900)
msn_serve_time_900 = {1: 9, 2: 8, 3: 10}
asn_1_serve_time_900 = {1: 3, 2: 4, 3: np.inf}
asn_2_serve_time_900 = {1: np.inf, 2: 5, 3: 4}

serve_times_900 = {
    ServerIDs.msn.value: msn_serve_time_900,
    ServerIDs.asn_1.value: asn_1_serve_time_900,
    ServerIDs.asn_2.value: asn_2_serve_time_900
}

# [900- 1100)
msn_serve_time_1100 = {1: 12, 2: 11, 3: 13}
asn_1_serve_time_1100 = {1: 4, 2: 5, 3: np.inf}
asn_2_serve_time_1100 = {1: np.inf, 2: 6, 3: 5}

serve_times_1100 = {
    ServerIDs.msn.value: msn_serve_time_1100,
    ServerIDs.asn_1.value: asn_1_serve_time_1100,
    ServerIDs.asn_2.value: asn_2_serve_time_1100
}

# [1100- 1500)
msn_serve_time_1500 = {1: 15, 2: 14, 3: 16}
asn_1_serve_time_1500 = {1: 5, 2: 6, 3: np.inf}
asn_2_serve_time_1500 = {1: np.inf, 2: 7, 3: 6}

serve_times_1500 = {
    ServerIDs.msn.value: msn_serve_time_1500,
    ServerIDs.asn_1.value: asn_1_serve_time_1500,
    ServerIDs.asn_2.value: asn_2_serve_time_1500
}

serve_times = {
    900: serve_times_900,
    1100: serve_times_1100,
    1500: serve_times_1500
}

In [24]:
# Develop a discrete event simulation...
results = handle_requests()

MSN
MSN Customers Length: 6170    
MSN
MSN Customers Length: 6170    
ASN1
ASN1 Customers Length: 2193   
ASN1
ASN1 Customers Length: 2193   
ASN2
ASN2 Customers Length: 767    
ASN2
ASN2 Customers Length: 767    


In [25]:
statistics = get_statistics(results)

In [26]:
statistics

{'MSN': {'mean': 16733.630621782853,
  'std': 21840.01484035462,
  'max': 68684.14229870639,
  'min': 8.818906654195967,
  'median': 15.154578688970762,
  'q25': 11.321050376473195,
  'q75': 35764.24033158924},
 'ASN1': {'mean': 1549.343874966719,
  'std': 3093.0105018773884,
  'max': 13255.296379840895,
  'min': 4.530996818419377,
  'median': 6.9236437812428715,
  'q25': 5.932571077501425,
  'q75': 1069.3334587469817},
 'ASN2': {'mean': 154.40893968537324,
  'std': 498.80468632628254,
  'max': 3833.2654724297718,
  'min': 5.557052967030876,
  'median': 7.023883842582672,
  'q25': 6.751453557221719,
  'q75': 7.551750669181507},
 1: {'mean': 10036.942630523778,
  'std': 18073.90313005564,
  'max': 67916.34335719985,
  'min': 4.530996818419377,
  'median': 12.940461351553674,
  'q25': 6.425701683912118,
  'q75': 10580.248757316276},
 2: {'mean': 12316.78508959237,
  'std': 19697.657613153613,
  'max': 68684.14229870639,
  'min': 5.698806065537793,
  'median': 12.268493374670644,
  'q25':

{'MSN': {'mean': 16733.630621782853,
  'std': 21840.01484035462,
  'max': 68684.14229870639,
  'min': 8.818906654195967,
  'median': 15.154578688970762,
  'q25': 11.321050376473195,
  'q75': 35764.24033158924},
 'ASN1': {'mean': 1549.343874966719,
  'std': 3093.0105018773884,
  'max': 13255.296379840895,
  'min': 4.530996818419377,
  'median': 6.9236437812428715,
  'q25': 5.932571077501425,
  'q75': 1069.3334587469817},
 'ASN2': {'mean': 154.40893968537324,
  'std': 498.80468632628254,
  'max': 3833.2654724297718,
  'min': 5.557052967030876,
  'median': 7.023883842582672,
  'q25': 6.751453557221719,
  'q75': 7.551750669181507},
 1: {'mean': 10036.942630523778,
  'std': 18073.90313005564,
  'max': 67916.34335719985,
  'min': 4.530996818419377,
  'median': 12.940461351553674,
  'q25': 6.425701683912118,
  'q75': 10580.248757316276},
 2: {'mean': 12316.78508959237,
  'std': 19697.657613153613,
  'max': 68684.14229870639,
  'min': 5.698806065537793,
  'median': 12.268493374670644,
  'q25':

In [27]:
#D = 10
#all_results = [handle_requests() for x in range(D)]